Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  regularizer = tf.nn.l2_loss(weights)
  loss = loss + (beta * regularizer)
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
num_steps = 801

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 51.319012
Training accuracy: 4.1%
Validation accuracy: 5.7%
Loss at step 100: 11.664075
Training accuracy: 74.3%
Validation accuracy: 72.3%
Loss at step 200: 4.420262
Training accuracy: 79.3%
Validation accuracy: 76.7%
Loss at step 300: 1.956985
Training accuracy: 82.7%
Validation accuracy: 79.8%
Loss at step 400: 1.120248
Training accuracy: 84.1%
Validation accuracy: 81.2%
Loss at step 500: 0.832859
Training accuracy: 84.6%
Validation accuracy: 81.8%
Loss at step 600: 0.732966
Training accuracy: 84.8%
Validation accuracy: 82.0%
Loss at step 700: 0.697917
Training accuracy: 84.9%
Validation accuracy: 82.0%
Loss at step 800: 0.685527
Training accuracy: 84.8%
Validation accuracy: 82.1%
Test accuracy: 88.9%


### With Neural Network:

In [8]:
batch_size = 128
hidden_unit = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_unit]))
  biases = tf.Variable(tf.zeros([hidden_unit]))
  weights1 = tf.Variable(
    tf.truncated_normal([hidden_unit, num_labels]))
  biases1 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  l = tf.matmul(tf_train_dataset, weights) + biases
  relu_layer = tf.nn.relu(l)
  logits = tf.matmul(relu_layer, weights1) + biases1
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  regularizer = tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights1)
  loss = loss + (beta * regularizer)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  l = tf.matmul(tf_valid_dataset, weights) + biases
  relu_layer = tf.nn.relu(l)
  logits = tf.matmul(relu_layer, weights1) + biases1
  valid_prediction = tf.nn.softmax(logits)
    
    
  l = tf.matmul(tf_test_dataset, weights) + biases
  relu_layer = tf.nn.relu(l)
  logits = tf.matmul(relu_layer, weights1) + biases1
  test_prediction = tf.nn.softmax(logits)

In [9]:
num_steps = 4501

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #print(offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3491.917969
Minibatch accuracy: 10.2%
Validation accuracy: 28.4%
Minibatch loss at step 500: 21.218357
Minibatch accuracy: 87.5%
Validation accuracy: 84.3%
Minibatch loss at step 1000: 0.947212
Minibatch accuracy: 80.5%
Validation accuracy: 82.9%
Minibatch loss at step 1500: 0.584535
Minibatch accuracy: 87.5%
Validation accuracy: 83.4%
Minibatch loss at step 2000: 0.609534
Minibatch accuracy: 89.8%
Validation accuracy: 83.4%
Minibatch loss at step 2500: 0.715374
Minibatch accuracy: 85.2%
Validation accuracy: 83.3%
Minibatch loss at step 3000: 0.768010
Minibatch accuracy: 82.0%
Validation accuracy: 83.4%
Minibatch loss at step 3500: 0.771944
Minibatch accuracy: 82.0%
Validation accuracy: 83.6%
Minibatch loss at step 4000: 0.666770
Minibatch accuracy: 86.7%
Validation accuracy: 84.0%
Minibatch loss at step 4500: 0.681977
Minibatch accuracy: 85.9%
Validation accuracy: 83.4%
Test accuracy: 90.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [10]:
num_steps = 4501

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
#     offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = (step * batch_size) % (500 - batch_size)
#     print(offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3409.614258
Minibatch accuracy: 13.3%
Validation accuracy: 19.5%
Minibatch loss at step 500: 21.083735
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Minibatch loss at step 1000: 0.494327
Minibatch accuracy: 99.2%
Validation accuracy: 79.2%
Minibatch loss at step 1500: 0.301410
Minibatch accuracy: 99.2%
Validation accuracy: 79.1%
Minibatch loss at step 2000: 0.283088
Minibatch accuracy: 100.0%
Validation accuracy: 79.3%
Minibatch loss at step 2500: 0.281859
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Minibatch loss at step 3000: 0.278243
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Minibatch loss at step 3500: 0.268233
Minibatch accuracy: 100.0%
Validation accuracy: 79.4%
Minibatch loss at step 4000: 0.269060
Minibatch accuracy: 100.0%
Validation accuracy: 79.1%
Minibatch loss at step 4500: 0.264563
Minibatch accuracy: 100.0%
Validation accuracy: 79.3%
Test accuracy: 86.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [11]:
batch_size = 128
hidden_unit = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_unit]))
  biases = tf.Variable(tf.zeros([hidden_unit]))
  weights1 = tf.Variable(
    tf.truncated_normal([hidden_unit, num_labels]))
  biases1 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  l = tf.matmul(tf_train_dataset, weights) + biases
  relu_layer = tf.nn.relu(l)
  keep_prob = tf.placeholder("float")
  relu_layer_drop = tf.nn.dropout(relu_layer, keep_prob)
  logits = tf.matmul(relu_layer_drop, weights1) + biases1
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  regularizer = tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights1)
  loss = loss + (beta * regularizer)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  l = tf.matmul(tf_valid_dataset, weights) + biases
  relu_layer = tf.nn.relu(l)
  logits = tf.matmul(relu_layer, weights1) + biases1
  valid_prediction = tf.nn.softmax(logits)
    
    
  l = tf.matmul(tf_test_dataset, weights) + biases
  relu_layer = tf.nn.relu(l)
  logits = tf.matmul(relu_layer, weights1) + biases1
  test_prediction = tf.nn.softmax(logits)

In [12]:
num_steps = 4501

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #print(offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3626.271729
Minibatch accuracy: 7.8%
Validation accuracy: 30.6%
Minibatch loss at step 500: 21.374718
Minibatch accuracy: 86.7%
Validation accuracy: 83.8%
Minibatch loss at step 1000: 1.028974
Minibatch accuracy: 78.1%
Validation accuracy: 83.0%
Minibatch loss at step 1500: 0.649347
Minibatch accuracy: 85.9%
Validation accuracy: 83.1%
Minibatch loss at step 2000: 0.687047
Minibatch accuracy: 86.7%
Validation accuracy: 83.2%
Minibatch loss at step 2500: 0.826526
Minibatch accuracy: 82.0%
Validation accuracy: 83.0%
Minibatch loss at step 3000: 0.824411
Minibatch accuracy: 80.5%
Validation accuracy: 83.1%
Minibatch loss at step 3500: 0.848144
Minibatch accuracy: 81.2%
Validation accuracy: 83.1%
Minibatch loss at step 4000: 0.742891
Minibatch accuracy: 84.4%
Validation accuracy: 83.5%
Minibatch loss at step 4500: 0.748224
Minibatch accuracy: 85.2%
Validation accuracy: 82.8%
Test accuracy: 89.7%


### Extreme overfitting:

In [13]:
num_steps = 4501

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
#     offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = (step * batch_size) % (500 - batch_size)
#     print(offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3623.282471
Minibatch accuracy: 9.4%
Validation accuracy: 35.8%
Minibatch loss at step 500: 21.156448
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 0.526479
Minibatch accuracy: 98.4%
Validation accuracy: 79.5%
Minibatch loss at step 1500: 0.336264
Minibatch accuracy: 99.2%
Validation accuracy: 79.3%
Minibatch loss at step 2000: 0.311579
Minibatch accuracy: 99.2%
Validation accuracy: 79.7%
Minibatch loss at step 2500: 0.317643
Minibatch accuracy: 99.2%
Validation accuracy: 79.8%
Minibatch loss at step 3000: 0.307327
Minibatch accuracy: 99.2%
Validation accuracy: 79.7%
Minibatch loss at step 3500: 0.289210
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Minibatch loss at step 4000: 0.281859
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Minibatch loss at step 4500: 0.288165
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Test accuracy: 86.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [23]:
batch_size = 128
beta = 1e-5

hidden_nodes_1 = 1024
hidden_nodes_2 = int(hidden_nodes_1 * 0.5)
hidden_nodes_3 = int(hidden_nodes_1 * np.power(0.5, 2))

graph = tf.Graph()
with graph.as_default():

    '''Input Data'''
    # For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    '''Variables'''
    # Hidden RELU layer 1
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_1], stddev=np.sqrt(2.0/(image_size*image_size))))
    biases_1 = tf.Variable(tf.zeros([hidden_nodes_1]))

    # Hidden RELU layer 2
    weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes_1, hidden_nodes_2], stddev=np.sqrt(2.0/hidden_nodes_1)))
    biases_2 = tf.Variable(tf.zeros([hidden_nodes_2]))
    
    # Hidden RELU layer 3
    weights_3 = tf.Variable(tf.truncated_normal([hidden_nodes_2, hidden_nodes_3], stddev=np.sqrt(2.0/hidden_nodes_2)))
    biases_3 = tf.Variable(tf.zeros([hidden_nodes_3]))
        
    # Output Layer
    weights_4 = tf.Variable(tf.truncated_normal([hidden_nodes_3, num_labels], stddev=np.sqrt(2.0/hidden_nodes_3)))
    biases_4 = tf.Variable(tf.zeros([num_labels]))
    
    # Hidden RELU layer 1
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    hidden_layer_1 = tf.nn.relu(logits_1)
    # Dropout on hidden layer: RELU layer
    keep_prob = tf.placeholder("float")
    hidden_layer_1_dropout = tf.nn.dropout(hidden_layer_1, keep_prob)
    
    
    # Hidden RELU layer 2
    logits_2 = tf.matmul(hidden_layer_1_dropout, weights_2) + biases_2
    hidden_layer_2 = tf.nn.relu(logits_2)
    # Dropout on hidden layer: RELU layer
    hidden_layer_2_dropout = tf.nn.dropout(hidden_layer_2, keep_prob)
    
    # Hidden RELU layer 3
    logits_3 = tf.matmul(hidden_layer_2_dropout, weights_3) + biases_3
    hidden_layer_3 = tf.nn.relu(logits_3)
    # Dropout on hidden layer: RELU layer
    hidden_layer_3_dropout = tf.nn.dropout(hidden_layer_3, keep_prob)
    
    # Output layer
    logits_4 = tf.matmul(hidden_layer_3_dropout, weights_4) + biases_4
    
    # Normal loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits_4, labels = tf_train_labels))
    # Loss function with L2 Regularization with decaying learning rate beta=0.5
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_4) 
    loss = tf.reduce_mean(loss + beta * regularizers)
    
    # Decaying learning rate
    global_step = tf.Variable(0)  # count the number of steps taken.
    start_learning_rate = 0.4
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training
    train_prediction = tf.nn.softmax(logits_4)
    
    # Predictions for validation 
    valid_logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    valid_relu_1 = tf.nn.relu(valid_logits_1)
    
    valid_logits_2 = tf.matmul(valid_relu_1, weights_2) + biases_2
    valid_relu_2 = tf.nn.relu(valid_logits_2)
    
    valid_logits_3 = tf.matmul(valid_relu_2, weights_3) + biases_3
    valid_relu_3 = tf.nn.relu(valid_logits_3)
    
    valid_logits_4 = tf.matmul(valid_relu_3, weights_4) + biases_4
        
    valid_prediction = tf.nn.softmax(valid_logits_4)
    
    # Predictions for test
    test_logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    test_relu_1 = tf.nn.relu(test_logits_1)
    
    test_logits_2 = tf.matmul(test_relu_1, weights_2) + biases_2
    test_relu_2 = tf.nn.relu(test_logits_2)
    
    test_logits_3 = tf.matmul(test_relu_2, weights_3) + biases_3
    test_relu_3 = tf.nn.relu(test_logits_3)
    
    test_logits_4 = tf.matmul(test_relu_3, weights_4) + biases_4
        
    test_prediction = tf.nn.softmax(test_logits_4)

In [24]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #print(offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.661196
Minibatch accuracy: 16.4%
Validation accuracy: 17.5%
Minibatch loss at step 2000: 0.435473
Minibatch accuracy: 89.8%
Validation accuracy: 86.5%
Minibatch loss at step 4000: 0.379866
Minibatch accuracy: 89.8%
Validation accuracy: 87.3%
Minibatch loss at step 6000: 0.568898
Minibatch accuracy: 82.0%
Validation accuracy: 87.9%
Minibatch loss at step 8000: 0.769313
Minibatch accuracy: 78.9%
Validation accuracy: 88.4%
Minibatch loss at step 10000: 0.411069
Minibatch accuracy: 87.5%
Validation accuracy: 89.1%
Minibatch loss at step 12000: 0.455144
Minibatch accuracy: 85.9%
Validation accuracy: 89.2%
Minibatch loss at step 14000: 0.433689
Minibatch accuracy: 86.7%
Validation accuracy: 89.5%
Minibatch loss at step 16000: 0.379443
Minibatch accuracy: 90.6%
Validation accuracy: 89.6%
Minibatch loss at step 18000: 0.356948
Minibatch accuracy: 90.6%
Validation accuracy: 89.4%
Minibatch loss at step 20000: 0.581993
Minibatch accuracy: 82.0%
Validation 